In [1]:
from pyabc import (ABCSMC, Distribution, RV,
                   MultivariateNormalTransition, AdaptivePNormDistance,
                  PNormDistance, History, AdaptivePopulationSize)
try:
    from pyabc.visualization import kde_1d
except:
    from pyabc.visualization.kde import kde_1d  # location for newer versions of pyabc
from pyabc.visualization import plot_kde_matrix
from pyabc.sampler import DaskDistributedSampler, SingleCoreSampler
from pyabc.transition import LocalTransition
import pyabc.visualization
import logging
import scipy as sp
import scipy.stats as st
import tempfile
import os
import math
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from dask.distributed import Client, LocalCluster
import csv


from tools.DataSimulator_BeeHaveV4_072020 import simulate

### Setting up the pyabc run

In [3]:
# Initialize the dask cluster 

#help(LocalCluster)

#c = LocalCluster(n_workers=(90/2), threads_per_worker = 2)  # Set for 96 vCPU compute instance

c = LocalCluster(n_workers = 2, threads_per_worker = 1)  # Set for 96 vCPU compute instance

client = Client(c,timeout=400)

C:\Users\TPurucke\miniconda3\envs\occupational-exposure\lib\site-packages\distributed\node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 64295 instead
  warnings.warn(


In [4]:
# Read the study data into a dictionary; note the outputs from model will need to look like this  and be named the same 
study_data_path = os.path.abspath('data/bee_counts/data_adults_eggs.csv')
reader = csv.reader(open(study_data_path, 'r'))
study_data = {}
for row in reader:
   k, v = row
   study_data[k] = float(v)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\git\\beehave_abc\\data\\bee_counts\\data_adults_eggs.csv'

Exception ignored in: <function Client.__del__ at 0x00000220FF49F430>
Traceback (most recent call last):
  File "C:\Users\TPurucke\miniconda3\envs\occupational-exposure\lib\site-packages\distributed\client.py", line 1189, in __del__
    self.close()
  File "C:\Users\TPurucke\miniconda3\envs\occupational-exposure\lib\site-packages\distributed\client.py", line 1408, in close
    for pc in self._periodic_callbacks.values():
AttributeError: 'Client' object has no attribute '_periodic_callbacks'


In [4]:
#Note, will add in more variables later, specifcally lifespan and egg laying rates. But for now, just vary the dose-response paramters. 
# Limits for the plots (accounts for the inverse log transform that pyabc will do with the results)
limits = dict(AdultAcutePar1=(0.0001, 0.1),
              AdultAcutePar2=(-9, -1),
              LarvaAcutePar1=(0.0001, 0.1),
              LarvaAcutePar2=(-9, -1))


priors = dict(AdultAcutePar1=(0.0001, 0.1),
              AdultAcutePar2=(-9, -1),
              LarvaAcutePar1=(0.0001, 0.1),
              LarvaAcutePar2=(-9, -1))

prior = Distribution(**{key: RV("uniform", a, b - a) #This creates a range
                        for key, (a,b) in priors.items()})



In [5]:
limits

{'AdultAcutePar1': (0.0001, 0.1),
 'AdultAcutePar2': (-9, -1),
 'LarvaAcutePar1': (0.0001, 0.1),
 'LarvaAcutePar2': (-9, -1)}

In [5]:
# Set our distance function and sampler
distance = pyabc.PNormDistance(p=1) #this is a distance function 

sampler = DaskDistributedSampler(dask_client = client) #This is distributes the jobs to the workers, I think? 

transition = LocalTransition() #local KDE fit following the move from one population to the next. You can specifiy different function si fyou want 
                               

In [6]:
# define the ABCSMC object
#Note that population size default is 100; that is, it will take 100 samples to build a population with before it moves on to the next particle 
#So, the total number of simulations would be the following calculation: First, take number of reps from a single parameterization that you need to get a sample, say 10 reps of each concentration(so, 10*6 = 160).
#, then, start 
#abc = ABCSMC(simulate, prior, distance,
#             population_size=2,
#             transitions = transition,
#             sampler=sampler)


abc = ABCSMC(simulate, prior, distance,
             population_size=2,
             transitions = transition)



#Note, Epsilon( the difference criterion) is assumed to be the default in this function(which is the median of the past population, I think); we'll have to look into that. 
#The goal is to make an number of sequential populations with increasingly relevant paramter values. You start with a uniform
#distribution as the prior, and then move to KDEs of the past populations. 

In [7]:
# Initialize a new ABC inference run
#db_path = ("sqlite:///" +
#           os.path.join(tempfile.gettempdir(), "test.db"))
database_dir = 'C:/Users/DDawso01/repos/ABCSQL'  # path for AWS EBS
#os.mkdir(database_dir)
db_path = ("sqlite:///" +
           os.path.join(database_dir, "run_test_updated.db"))
abc.new(db_path, study_data)

INFO:History:Start <ABCSMC(id=2, start_time=2020-07-30 11:51:40.587080, end_time=None)>


2

In [8]:
history = abc.run(max_nr_populations=2, minimum_epsilon=0.5) #

INFO:Epsilon:initial epsilon is 2371580.1155399857
INFO:ABC:t:0 eps:2371580.1155399857
INFO:ABC:t:1 eps:2291662.845159529


KeyboardInterrupt: 

In [9]:
pop_data = history.get_all_populations()
pop_data

NameError: name 'history' is not defined

In [ ]:
#get acceptance rates for each generation
print("Acceptance rates")
for gen, samples, particles in zip(pop_data['t'],pop_data['samples'], pop_data['particles']):
    if(samples>0):
        print("Gen {}: {:.1f}%".format(gen+1,particles/samples*100))

In [ ]:
history.total_nr_simulations

### Visualize results

In [ ]:
df, w = history.get_distribution(m=0, t=history.max_t)
plot_kde_matrix(df, w, limits=limits);

In [ ]:
variable = 'AIAdultLD50'
fig, ax = plt.subplots()
for t in range(0,history.max_t+1):
    df, w = history.get_distribution(m=0, t=t)
    pyabc.visualization.plot_kde_1d(df, w, xmin=0.0001, xmax=0.1,
                                   x=variable, ax=ax,
                                   label="PDF t={}".format(t))
x, pdf = kde_1d(df,w,variable, numx = 200)
MAP = x[np.where(np.max(pdf)==pdf)]
plt.axvline(x=MAP.item(),color='black')
ax.legend()
ax.annotate('Max: {:.2f} ng/bee'.format(MAP.item()), xy = (0.05,0.75))
plt.show()

In [ ]:
variable = 'ICForagerLifespan_mean'
fig, ax = plt.subplots()
for t in range(0,history.max_t + 1,1):
    df, w = history.get_distribution(m=0, t=t)
    pyabc.visualization.plot_kde_1d(df, w, xmin=4, xmax=16,
                                   x=variable, ax=ax,
                                   label="PDF t={}".format(t))
x, pdf = kde_1d(df,w,variable, numx = 200)
MAP = x[np.where(np.max(pdf)==pdf)]
ax.legend()
plt.axvline(x=MAP.item(),color='black')
#ax.annotate('Max: {:.2f} days'.format(MAP.item()), xy = (5.5,0.4))
ax.legend()
plt.show()

### Visualize from a stored run

In [10]:
#retrieve stored run - run ID 1
#database_dir = 'results_db/'
#db_path = ("sqlite:///" +
#           os.path.join(database_dir, "run_11_18.db"))
history = History(db_path)
#history.id = 1

In [ ]:
#df, w = history.get_distribution(m=0, t=history.max_t)

In [ ]:
#plot_kde_matrix(df, w, limits=limits);

In [11]:
history.get_all_populations()

,t,population_end_time,samples,epsilon,particles
0,-1,2020-07-30 11:51:44.344273,2,inf,1
1,0,2020-07-30 12:46:41.100367,11,2.371580e+06,2


### Continuing a stopped run

In [ ]:
#retrieve stored run 
database_dir = 'results_db/'
db_path = ("sqlite:///" +
           os.path.join(database_dir, "run_3_11_2.db"))
#db = History(db_path)
#db.id = 3

In [ ]:
# define the ABCSMC object
#abc_cont = ABCSMC(simulate, prior, distance,
#             population_size=AdaptivePopulationSize(100, max_population_size = 100),
#             transitions = transition,
#             sampler=sampler)

In [ ]:
#abc_cont.load(db_path,1)

In [ ]:
#abc_cont.run(minimum_epsilon=0, max_nr_populations=100)